In [1]:
%pip install openai

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/567.4 kB ? eta -:--:--
   ---------------------------------------- 567.4/567.4 kB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------------- -- 1.8/2.0 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 8.5 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
Note: you may need to restart the kernel to use updated packages.


In [6]:
from openai import OpenAI
import config

In [7]:
client = OpenAI(
    api_key=config.custom_api_key
)

In [8]:
def chat_with_gpt(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

In [ ]:
# Text example
user_input = "What is the capital of Bosnia and Herzegovina" 
response = chat_with_gpt(user_input)
print(f"Chatbot: {response}")

Chatbot: The capital of Bosnia and Herzegovina is Sarajevo.


In [ ]:
# Image example
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

response = client.chat.completions.create(
    model="gpt-4o", 
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "What's in this image?"},
            {
                "type": "image_url", 
                "image_url": {
                    "url": image_url,
                },
            },
        ],
    }],
)

print(response.choices[0].message.content)

The image depicts a scenic landscape with a wooden boardwalk leading through a lush green field. The sky is blue with scattered clouds, creating a serene and open atmosphere. The surrounding area is filled with tall grass and some bushes, suggesting a natural or park setting.


In [9]:
# Prompt templates
import utils
import base64
import os

system_prompt = """You are a UX expert for web applications.
Your task is to identify usability issues with the
information you get for an application’s view.
An example of a usability issue could be: ‘Lack of
visual feedback on user interactions’.
Respond using app domain language; you must not use
technical terminology or mention code details.
Enumerate the problems identified; add an empty
paragraph after each enumeration; no preceding
or following text."""

app_overview = utils.app_overview
user_task = utils.user_task
source_code = utils.source_code

user_prompt = f"""I have a python web application written in Django about: {app_overview}
The user’s task in this app view is about: {user_task}.
An image of the app view is provided.
Below is the incomplete code for the app view.
This code includes the view’s user interface.
Source Code:
{source_code}"""


In [10]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def detect_usability_issues(image_filename):
    image_path = os.path.join(os.pardir, "images", image_filename) 
    
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Error: The file '{image_path}' was not found. Check the path.")

    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}",
                        },
                    },
                ],
            },
        ],
    )

    print(response.choices[0].message.content)

In [16]:
# Example usage
detect_usability_issues("all_courses.png") 

1. **Lack of Visual Feedback for Links and Buttons**: The "View" buttons may not provide enough visual feedback on interaction, such as a hover effect to indicate clickability.

2. **Inconsistent Image Aspect Ratio**: Course images could have inconsistent aspect ratios, affecting the overall alignment and visual consistency.

3. **Text Alignment and Spacing**: Card text description spacing seems inconsistent, which can make reading more difficult.

4. **Responsive Design Issues**: It is unclear if the cards adjust well on different screen sizes due to fixed dimensions within inline styles.

5. **Overuse of Inline Styles**: The extensive use of inline styles can hinder design adjustments and maintenance.

6. **Insufficient Contrast for Text**: The contrast between text and background might not be adequate for readability in some cases, especially for accessibility.

7. **Limited Course Information Display**: The course names and summaries may truncate important information, limiting use

In [11]:
# Addition of the Nielsen's 10 general principles for interaction design

system_prompt = f"""You are a UX expert for web applications. Your task is to identify usability
issues with the information you get for an application’s view. For the identification of usability issues,
you should use the following Nielsen’s usability principles:

NS01—Visibility of system status;
NS02—User control and freedom;
NS03—Help users recognize, diagnose, and recover from from errors;
NS04—Consistency and standards;
NS05—Recognition rather than recall;
NS06—Match Between System and The Real World;
NS07—Error Prevention;
NS08—Flexibility and Efficiency of Use;
NS09—Aesthetic and Minimalist Design;
NS10—Help and Documentation.

A description of the Nielsen's usability principles is given in {utils.nielsen_principles}.

Based on the above information, analyze and evaluate the website’s performance referencing the information you get for an application’s view,
which includes the image of the web app, code, app overview and user task, according to Nielsen’s usability
principles. Rate each area on an integer scale from e to 5, adjusting the score up or down
from a baseline of 3 points based on significant differences. Finally, provide justifications
for each rating and summarize the overall evaluation.
Respond using app domain language; you must not use
technical terminology or mention code details.
Enumerate the problems identified; add an empty
paragraph after each enumeration; no preceding
or following text."""

app_overview = utils.app_overview
user_task = utils.user_task
source_code = utils.source_code

user_prompt = f"""I have a web application written about: {app_overview}
The user’s task in this app view is about: {user_task}.
An image of the app view is provided.
Below is the incomplete code for the app view.
This code includes the view’s user interface.
Source Code:
{source_code}"""


In [12]:
# Example usage
detect_usability_issues("all_courses.png") 

Evaluation of the web application's view on Nielsen's usability principles:

1. **Visibility of System Status (NS01):** 4/5

   The system status is well conveyed by displaying available courses clearly with images, titles, and descriptions. However, there is no loading indicator for cases when the list is being fetched which could enhance status visibility.

2. **User Control and Freedom (NS02):** 3/5

   Users can view courses through clearly marked buttons, but there is no visible option to filter or sort courses, which could provide better control and freedom in navigation.

3. **Help Users Recognize, Diagnose, and Recover from Errors (NS03):** 3/5

   The message "There are no courses yet!" is clear if no courses are available. However, there could be more guidance on refreshing or checking again later to improve error recovery.

4. **Consistency and Standards (NS04):** 4/5

   The design is consistent in terms of layout and button styling. However, the use of different column cla